In [2]:
import numpy as np 
import pandas as pd
import sklearn 
import tensorflow as tf 
import nltk
import gensim 
import json
import re
import string
import contractions

In [55]:
##TO-DO: Read dataset.json as a dictionary object 
with open('dataset.json', 'r') as file:
    json_str = file.read()
    json_dic = json.loads(json_str)


In [56]:
##TO-DO: Convert dictionary object to pandas dataframe 
df = pd.DataFrame.from_dict(json_dic).T
df['post_tokens_string'] = df['post_tokens'].apply(lambda x: ' '.join(x))
df.head()

,post_id,annotators,rationales,post_tokens,post_tokens_string
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ...",i dont think im getting my baby them white 9 h...
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem...",we cannot continue calling ourselves feminists...
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]",nawt yall niggers ignoring me
1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[<user>, i, am, bit, confused, coz, chinese, p...",<user> i am bit confused coz chinese ppl can n...
1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[this, bitch, in, whataburger, eating, a, burg...",this bitch in whataburger eating a burger with...


### Pre-Process Data

In [57]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

# download a create set of stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words("english"))

# exclude set contains punctuation
exclude = set(string.punctuation)

# create lemmatizer
lemmatizer = WordNetLemmatizer()

#     # Paramaters: text_features: text (tweet) data in our dataframe.
#     # When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training
 
def clean_text(text_features):
    # tokenize and lower case
    tokens = word_tokenize(text_features.lower())

    # remove stopwords
    tokens = [x for x in tokens if x not in stopwords]

    # remove punctuation 
    tokens = [token for token in tokens if token not in exclude]

    # remove non alphabetical characters
    tokens = [x for x in tokens if any(char.isalpha() for char in x)]

    # undo contractions
    tokens = [contractions.fix(token) for token in tokens]

    # lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens 

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


In [58]:
from collections import Counter


df['post_tokens_cleaned'] = df['post_tokens_string'].apply(clean_text)

def majority_label(annotations):
    labels = [annotation['label'] for annotation in annotations]
    counter = Counter(labels)
    majority_label = counter.most_common(1)[0][0]
    return 0 if majority_label == 'normal' else 1

df['label'] = df['annotators'].apply(majority_label)

df.head()

,post_id,annotators,rationales,post_tokens,post_tokens_string,post_tokens_cleaned,label
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[i, dont, think, im, getting, my, baby, them, ...",i dont think im getting my baby them white 9 h...,"[do not, think, i am, getting, baby, white, tw...",0
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],"[we, cannot, continue, calling, ourselves, fem...",we cannot continue calling ourselves feminists...,"[continue, calling, feminist, right, womxn, ar...",0
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],"[nawt, yall, niggers, ignoring, me]",nawt yall niggers ignoring me,"[nawt, you all, nigger, ignoring]",0
1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[<user>, i, am, bit, confused, coz, chinese, p...",<user> i am bit confused coz chinese ppl can n...,"[user, bit, confused, coz, chinese, people, ac...",1
1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[this, bitch, in, whataburger, eating, a, burg...",this bitch in whataburger eating a burger with...,"[bitch, whataburger, eating, burger, top, bun,...",1


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

"""
    use Word2Vec or Sentence2Vec to vectorize the cleaned text data
 
    Paramaters
    ---------
    cleaned_text: (cleaned) text (tweet) data in our dataframe.
    """
    # When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training
    # Later: Sentence2Vec or Doc2Vec

def flatten(l):
        return [item for sublist in l for item in sublist]

def vectorize_text(cleaned_text, use_word2vec=True):
    # Initialize Word2Vec model
    model = Word2Vec(cleaned_text, min_count=1)
    
    # Initialize feature matrix
    feature_matrix = []

    # Iterate over each sentence in cleaned text
    for sentence in cleaned_text:

        # Initialize vectors and word count
        vectors = []
        word_count = len(sentence)

        # Generate word vectors for each word in sentence
        for word in sentence:
             vectors.append(model.wv[word].tolist())

        # Initialize averaged vector
        averaged_vector = []

        # Calculate average vector for each word in sentence
        for i in range(0, len(vectors[0])):
            count = 0
            
            for v in vectors:
                count += v[i]
                
            avg = count/word_count
            
            averaged_vector.append(avg)
            
        # Append averaged vector to feature matrix
        feature_matrix.append(averaged_vector)

    # Return feature matrix
    return feature_matrix

In [60]:
feature_matrix = vectorize_text(df['post_tokens_cleaned'])

print(len(feature_matrix), len(feature_matrix[0]))

20148 100


### Model Training: KNN/Random Forest/RNNs

In [64]:
from sklearn.model_selection import train_test_split

## TO-DO: split our data into train and test data
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, df["label"].values)

15111


##### Using KNN
###### Refer to this resource: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [47]:
## TO-DO: Determine initial value of parameters (especially K)
k = 0

## TO-DO: determine the best distance computation metric
candidate_metrics = ['euclidean','cosine','manhattan']
best_metric = ''




In [48]:
from sklearn.neighbors import KNeighborsClassifier

## TO-DO: fit a KNN Classifier with distance metric and K 

In [49]:
# TO-DO: calculating accuracy
from sklearn.metrics import accuracy_score


In [50]:
## TO-DO: optimize K with hyperparameter tuning

from sklearn.model_selection import cross_val_score


## TO-DO: Visualize results to determine best K


In [51]:
## TO-DO: repeat training with optimized K

##### Using XGBoost/Random Forest
###### Refer to this resource: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [52]:
## TO-DO: Train model naively, with default parameters
from sklearn.ensemble import RandomForestClassifier

In [53]:
## TO-DO: Evaluate base model
from sklearn.metrics import accuracy_score



In [54]:
##TO-DO: Hyperparameter Tuning via RandomizedSearch & GridSearch 

## get current parameters and their values 
from pprint import pprint


## Randomized Search 
from sklearn.model_selection import RandomizedSearchCV
random_grid = {}
rf_random = RandomizedSearchCV()

## Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {}
grid_search = GridSearchCV()

TypeError: RandomizedSearchCV.__init__() missing 2 required positional arguments: 'estimator' and 'param_distributions'

##### Using LSTMs
###### Refer to this resource: https://www.youtube.com/watch?v=8HyCNIVRbSU&ab_channel=TheA.I.Hacker-MichaelPhi

In [ ]:
from tensorflow.python.keras.layers import LSTM, Activation, Dropout, Dense, Embedding
from tensorflow.python.keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences


In [ ]:
## veryyyyyy basic architecture. TO-DO: complete and expand

model = Sequential()
model.add(Embedding())
model.add(LSTM())
model.add(Dense(2, activation='softmax'))

model.compile(loss='',optimizer='', metrics=['accuracy'])

In [ ]:
epochs = 0
batch_size = 0

## TO-DO: determine training parameters and complete call
model.fit()

In [ ]:
## TO-DO: evaluate function 
model.evaluate()

### Using BERT
###### Refer to: http://jalammar.github.io/illustrated-bert/ 
###### Refer to: https://medium.com/@parthdholakiya180/twitter-hate-detection-using-bert-e7682b2d0a0c 

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer